In [1]:
! pip install langchain langchain-community langchain-text-splitters langchain-huggingface sentence-transformers faiss-cpu langchain-groq -q
! pip install langchain-core langchain-ollama pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 82.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
from google.colab import userdata
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=GROQ_API_KEY, model="Llama3-8b-8192")

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the provided context only.
    Please provide the most accurate response based on the question
    <context>
    {context}
    <context>
    Question:{input}
    """
)

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [6]:
def create_vector_embedding():
      embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
      loader=PyPDFDirectoryLoader("/content/sample_data/research_papers") #Data ingestion
      docs=loader.load() #Document loading
      text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
      final_document=text_splitter.split_documents(docs)
      vectors=FAISS.from_documents(final_document,embeddings)
      return vectors

In [7]:
if __name__=="__main__":
    print("vector embedding started")
    vectors = create_vector_embedding()
    print("vector embedding completed")

    document_chain=create_stuff_documents_chain(llm=llm, prompt=prompt)
    retriever=vectors.as_retriever()
    retrival_chain=create_retrieval_chain(retriever,document_chain)

vector embedding started


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

vector embedding completed


In [8]:
response=retrival_chain.invoke({'input':'what is llm?'})
print(response['answer'])

Based on the provided context, it appears that LLM refers to "Large Language Models".


In [9]:
response1=retrival_chain.invoke({'input':'what is Data Parallelism?'})
print(response1['answer'])

Based on the provided context, Data Parallelism is described as:

"Data parallelism replicates the model on multiple devices where data in a batch gets divided across devices. At the end of each training iteration weights are synchronized across all devices."


In [10]:
response2=retrival_chain.invoke({'input':'what is Transformers?'})
print(response2['answer'])

Based on the provided context, it appears that the question is asking about the concept of "Transformers" in the context of natural language processing.


In [11]:
response3=retrival_chain.invoke({'input':'what is positional encoding?'})
print(response3['answer'])

Based on the provided context, positional encoding refers to the technique introduced in the transformer architecture to capture positional information. This is necessary because the attention mechanism in the transformer does not capture positional information, and as a result, positional embeddings were introduced. Variants of positional embedding include absolute, relative, or learned positional encodings, and within relative encoding, Alibi and RoPE are two widely used positional embeddings in LLMs.
